In [7]:
from PIL import Image, ImageDraw, ImageFont

def create_text_box_image(text, output_path="output_image.png"):
    # Размер изображения (например, 1920x1080 для экрана Full HD)
    screen_width = 1920
    screen_height = 1080

    # Задаем ширину окошка (1/4 ширины экрана)
    box_width = screen_width // 4

    # Задаем начальные параметры
    padding = 20  # Отступы внутри окошка
    font_size = 24
    font = ImageFont.truetype("arial.ttf", font_size)  # Шрифт

    # Создаем пустое изображение
    image = Image.new("RGB", (screen_width, screen_height), (173, 216, 230))  # Цвет фона (светло-зеленый)
    draw = ImageDraw.Draw(image)

    # Определяем размеры текста
    text_width, text_height = draw.multiline_textsize(text, font=font)
    box_height = text_height + 2 * padding

    # Координаты центра
    box_x = (screen_width - box_width) // 2
    box_y = (screen_height - box_height) // 2

    # Рисуем окошко
    draw.rectangle(
        [(box_x, box_y), (box_x + box_width, box_y + box_height)],
        fill="white",
        outline="black",
        width=2,
    )

    # Пишем текст внутри окошка
    text_x = box_x + padding
    text_y = box_y + padding
    draw.multiline_text((text_x, text_y), text, fill="black", font=font, align="left")

    # Сохраняем изображение
    image.save(output_path)
    print(f"Изображение сохранено: {output_path}")


# Пример текста
example_text = "в анкете предварительной записи репетитора французского была графа «какая у вас цель». написала что мой будущий муж паша дуров непонятно насколько будет находиться во франции, мне нужно быть готовой"

# Создание изображения
create_text_box_image(example_text)


OSError: cannot open resource

In [19]:
from PIL import Image, ImageDraw, ImageFont

def create_telegram_style_message(text, output_path="output_image.png"):
    screen_width = 1920
    screen_height = 1080
    box_width = screen_width // 3
    padding = 30
    font_size = 60
    corner_radius = 20

    # Попытка загрузить шрифт
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except OSError:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", font_size)
        except OSError:
            font = ImageFont.load_default()
            print("Используется шрифт по умолчанию, который может не поддерживать кириллицу.")

    # Создаем изображение
    image = Image.new("RGB", (screen_width, screen_height), (173, 216, 230))  # Цвет фона
    draw = ImageDraw.Draw(image)

    # Вычисляем размеры текста с учетом шрифта
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    # Высота и ширина окна
    box_height = text_height + 2 * padding
    box_x = (screen_width - box_width) // 2
    box_y = (screen_height - box_height) // 2

    # Координаты текста (внутри окна)
    text_x = box_x + padding
    text_y = box_y + padding

    # Рисуем закругленный прямоугольник
    draw.rounded_rectangle(
        [(box_x, box_y), (box_x + box_width, box_y + box_height)],
        radius=corner_radius,
        fill="white",
        outline="black",
        width=2,
    )

    # Рисуем текст
    draw.text((text_x, text_y), text, fill="black", font=font, align="left")

    # Сохраняем изображение
    image.save(output_path)
    print(f"Изображение сохранено: {output_path}")


# Пример текста
example_text = "Hello World"

# Создание изображения
create_telegram_style_message(example_text)


Используется шрифт по умолчанию, который может не поддерживать кириллицу.
Изображение сохранено: output_image.png


In [17]:
def open_tgv_file(file_path):
    try:
        # Открытие файла в текстовом режиме
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.read()
        print("Содержимое файла:")
        print(data)
    except UnicodeDecodeError:
        print("Файл не является текстовым или имеет нестандартную кодировку.")
    except Exception as e:
        print(f"Ошибка при открытии файла: {e}")

# Пример использования
file_path = "background.tgv"  # Укажите путь к вашему файлу
open_tgv_file(file_path)


Файл не является текстовым или имеет нестандартную кодировку.


AttributeError: 'FreeTypeFont' object has no attribute 'getsize'

In [27]:
from PIL import Image, ImageDraw, ImageFont

def add_text_overlay(image_path, output_path, text):
    # Загрузить изображение
    image = Image.open(image_path)
    image_width, image_height = image.size

    # Настройки окна с текстом
    window_width = image_width // 3
    font_size = 20

    # Попытка загрузить шрифт
    try:
        font = ImageFont.truetype("OpenSans.ttf", font_size)
    except OSError:
        font = ImageFont.load_default()  # Загрузить шрифт по умолчанию, если arial недоступен

    # Разбить текст на строки, которые помещаются в ширину окна
    lines = []
    words = text.split()
    current_line = ""

    for word in words:
        test_line = f"{current_line} {word}".strip()
        test_bbox = font.getbbox(test_line)  # Получаем размеры текста
        test_width = test_bbox[2] - test_bbox[0]  # Вычисляем ширину текста

        if test_width <= window_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word

    if current_line:
        lines.append(current_line)

    # Вычислить высоту окна с текстом
    line_height = font_size + 5  # Добавим небольшой интервал между строками
    window_height = line_height * len(lines)

    # Создать новое изображение с белым фоном для окна с текстом
    text_window = Image.new("RGBA", (window_width, window_height), "white")
    draw = ImageDraw.Draw(text_window)

    # Отрисовать текст в окне
    for i, line in enumerate(lines):
        draw.text((5, i * line_height), line, font=font, fill="black")

    # Вычислить позицию для вставки окна с текстом
    position_x = (image_width - window_width) // 2
    position_y = 0

    # Добавить окно с текстом на оригинальное изображение
    image.paste(text_window, (position_x, position_y), text_window)

    # Сохранить результат
    image.save(output_path)

# Использование функции
image_path = "theme_tg.png"  # Замените на путь к вашему изображению
output_path = "output.jpg"  # Путь для сохранения результата
text = "В результате недавних погодных явлений, местные жители столкнулись с временными неудобствами, связанными с повышением уровня воды. Однако, традиционные практики и обычаи помогают сообществу адаптироваться к изменившимся условиям"

add_text_overlay(image_path, output_path, text)


In [51]:
from PIL import Image, ImageDraw, ImageFont

def add_text_overlay(image_path, output_path, text):
    # Загрузить изображение
    image = Image.open(image_path)
    image_width, image_height = image.size

    # Настройки окна с текстом
    window_width = image_width // 3
    font_size = 12
    corner_radius = 10  # Радиус закругленных углов
    padding = 20  # Отступы текста от краёв окна

    # Попытка загрузить шрифт
    try:
        font = ImageFont.truetype("OpenSans.ttf", font_size)
    except OSError:
        font = ImageFont.load_default()  # Загрузить шрифт по умолчанию, если шрифт недоступен

    # Разбить текст на строки, которые помещаются в ширину окна
    lines = []
    words = text.split()
    current_line = ""

    # Реальная ширина текста, учитывающая отступы
    max_text_width = window_width - 2 * padding

    for word in words:
        test_line = f"{current_line} {word}".strip()
        test_bbox = font.getbbox(test_line)  # Получаем размеры текста
        test_width = test_bbox[2] - test_bbox[0]  # Вычисляем ширину текста

        if test_width <= max_text_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word

    if current_line:
        lines.append(current_line)

    # Вычислить высоту окна с текстом
    line_height = font_size + 5  # Добавим небольшой интервал между строками
    window_height = line_height * len(lines) + 2 * padding

    # Создать изображение для окна
    text_window = Image.new("RGBA", (window_width, window_height + padding), (255, 255, 255, 0))
    draw = ImageDraw.Draw(text_window)

    # Рисуем закругленный прямоугольник
    rect_x0, rect_y0 = 0, 20  # Верхний левый угол (20 пикселей для треугольника)
    rect_x1, rect_y1 = window_width, window_height  # Нижний правый угол
    draw.rounded_rectangle([(rect_x0, rect_y0), (rect_x1, rect_y1)], radius=corner_radius, fill="white")

    # Добавляем треугольник слева сверху
    triangle = [(10, 0), (30, 20), (0, 20)]  # Точки треугольника
    draw.polygon(triangle, fill="white")

    # Отрисовать текст в окне с учётом отступов
    for i, line in enumerate(lines):
        draw.text((padding, rect_y0 + padding + i * line_height), line, font=font, fill="black")

    # Вычислить позицию для вставки окна с текстом
    position_x = (image_width - window_width) // 2
    position_y = int(image_height * 0.1)  # Смещение вниз на 10% от высоты изображения

    # Добавить окно с текстом на оригинальное изображение
    image.paste(text_window, (position_x, position_y), text_window)

    # Сохранить результат
    image.save(output_path)

# Использование функции
image_path = "theme_tg.png"  # Замените на путь к вашему изображению
output_path = "output3333.jpg"  # Путь для сохранения результата
text = "В результате недавних погодных явлений, местные жители столкнулись с временными неудобствами, связанными с повышением уровня воды. Однако, традиционные практики и обычаи помогают сообществу адаптироваться к изменившимся условиям"

add_text_overlay(image_path, output_path, text)


In [55]:
from PIL import Image, ImageDraw, ImageFont

def add_text_overlay(image_path, output_path, text):
    # Загрузить изображение
    image = Image.open(image_path)
    image_width, image_height = image.size

    # Настройки окна с текстом
    window_width = image_width // 3
    font_size = 14
    corner_radius = 20  # Радиус закругленных углов
    padding = 20  # Отступы сверху, слева и справа
    bottom_padding = 30  # Увеличенный отступ снизу

    # Попытка загрузить шрифт
    try:
        font = ImageFont.truetype("OpenSans.ttf", font_size)
    except OSError:
        font = ImageFont.load_default()  # Загрузить шрифт по умолчанию, если шрифт недоступен

    # Разбить текст на строки, которые помещаются в ширину окна
    lines = []
    words = text.split()
    current_line = ""

    # Реальная ширина текста с учётом отступов
    max_text_width = window_width - 2 * padding

    for word in words:
        test_line = f"{current_line} {word}".strip()
        test_bbox = font.getbbox(test_line)  # Получаем размеры текста
        test_width = test_bbox[2] - test_bbox[0]  # Вычисляем ширину текста

        if test_width <= max_text_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word

    if current_line:
        lines.append(current_line)

    # Вычислить высоту окна с учётом нижнего паддинга
    line_height = font_size + 5  # Добавим небольшой интервал между строками
    window_height = line_height * len(lines) + padding + bottom_padding

    # Создать изображение для окна
    text_window = Image.new("RGBA", (window_width, window_height + padding), (255, 255, 255, 0))
    draw = ImageDraw.Draw(text_window)

    # Рисуем закругленный прямоугольник
    rect_x0, rect_y0 = 0, 20  # Верхний левый угол (20 пикселей для треугольника)
    rect_x1, rect_y1 = window_width, window_height  # Нижний правый угол
    draw.rounded_rectangle([(rect_x0, rect_y0), (rect_x1, rect_y1)], radius=corner_radius, fill="white")

    # Добавляем треугольник слева сверху
    triangle = [(10, 0), (30, 20), (0, 20)]  # Точки треугольника
    draw.polygon(triangle, fill="white")

    # Отрисовать текст в окне с учётом отступов
    for i, line in enumerate(lines):
        draw.text((padding, rect_y0 + padding + i * line_height), line, font=font, fill="black")

    # Вычислить позицию для вставки окна с текстом
    position_x = (image_width - window_width) // 2
    position_y = int(image_height * 0.1)  # Смещение вниз на 10% от высоты изображения

    # Добавить окно с текстом на оригинальное изображение
    image.paste(text_window, (position_x, position_y), text_window)

    # Сохранить результат
    image.save(output_path)

# Использование функции
image_path = "theme_tg.png"  # Замените на путь к вашему изображению
output_path = "output.jpg"  # Путь для сохранения результата
text = "В результате недавних погодных явлений, местные жители столкнулись с временными неудобствами, связанными с повышением уровня воды. Однако, традиционные практики и обычаи помогают сообществу адаптироваться к изменившимся условиям"

add_text_overlay(image_path, output_path, text)


In [23]:
from PIL import Image, ImageDraw, ImageFont

def add_text_overlay(image_path, output_path, text):
    # Загрузить изображение
    image = Image.open(image_path).convert("RGBA")  # Работа с прозрачностью
    image_width, image_height = image.size

    # Настройки окна с текстом
    window_width = image_width // 3
    font_size = 14  # Увеличенный шрифт для улучшения читабельности
    corner_radius = 20  # Радиус закругленных углов
    top_padding = 30  # Отступ сверху
    side_padding = 20  # Отступы слева и справа
    bottom_padding = 20  # Отступ снизу

    # Загрузка шрифта
    try:
        font = ImageFont.truetype("OpenSans.ttf", font_size)
    except OSError:
        font = ImageFont.load_default()

    # Разбить текст на строки
    lines = []
    words = text.split()
    current_line = ""
    max_text_width = window_width - 2 * side_padding

    for word in words:
        test_line = f"{current_line} {word}".strip()
        test_width = font.getbbox(test_line)[2]  # Ширина строки

        if test_width <= max_text_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word
    if current_line:
        lines.append(current_line)

    # Высота окна текста
    line_height = font.getbbox("hg")[3] + 5
    window_height = line_height * len(lines) + top_padding + bottom_padding

    # Создание изображения для текста
    text_window = Image.new("RGBA", (window_width, window_height), (255, 255, 255, 0))
    draw = ImageDraw.Draw(text_window)

    # Рисуем закругленный прямоугольник
    draw.rounded_rectangle(
        [(0, 0), (window_width, window_height)],
        radius=corner_radius,
        fill=(255, 255, 255, 255),  # Полупрозрачный белый
    )

    # Добавляем текст в окно
    for i, line in enumerate(lines):
        draw.text(
            (side_padding, top_padding + i * line_height),
            line,
            font=font,
            fill=(0, 0, 0, 255),  # Чёрный текст
        )

    # Позиция для текста
    position_x = (image_width - window_width) // 2
    position_y = int(image_height * 0.1)

    # Объединяем изображения
    combined = Image.alpha_composite(image, Image.new("RGBA", image.size, (255, 255, 255, 0)))
    combined.paste(text_window, (position_x, position_y), text_window)

    # Сохранить результат
    combined.convert("RGB").save(output_path, "JPEG", quality=95)

# Использование функции
image_path = "theme_tg.png"
output_path = "output7776.jpg"
text = "В результате недавних погодных явлений, местные жители столкнулись с временными неудобствами, связанными с повышением уровня воды. Однако, традиционные практики и обычаи помогают сообществу адаптироваться к изменившимся условиям."

add_text_overlay(image_path, output_path, text)


In [27]:
from PIL import Image, ImageDraw, ImageFont

def add_text_overlay(image_path, output_path, text):
    # Загрузить изображение
    image = Image.open(image_path).convert("RGBA")  # Работа с прозрачностью
    image_width, image_height = image.size

    # Настройки окна с текстом
    window_width = image_width // 3
    font_size = 14  # Увеличенный шрифт для улучшения читабельности
    corner_radius = 20  # Радиус закругленных углов
    top_padding = 30  # Отступ сверху
    side_padding = 20  # Отступы слева и справа
    bottom_padding = 20  # Отступ снизу

    # Загрузка шрифта
    try:
        font = ImageFont.truetype("OpenSans.ttf", font_size)
    except OSError:
        font = ImageFont.load_default()

    # Разбить текст на строки
    lines = []
    words = text.split()
    current_line = ""
    max_text_width = window_width - 2 * side_padding

    for word in words:
        test_line = f"{current_line} {word}".strip()
        test_width = font.getbbox(test_line)[2]  # Ширина строки

        if test_width <= max_text_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word
    if current_line:
        lines.append(current_line)

    # Высота окна текста
    line_height = font.getbbox("hg")[3] + 5
    window_height = line_height * len(lines) + top_padding + bottom_padding

    # Создание изображения для текста
    text_window = Image.new("RGBA", (window_width, window_height), (255, 255, 255, 0))
    draw = ImageDraw.Draw(text_window)

    # Рисуем прямоугольник с закругленными углами (кроме левого верхнего)
    # Начнем с рисования обычного прямоугольника с закругленными углами
    draw.rounded_rectangle(
        [(0, 0), (window_width, window_height)],
        radius=corner_radius,
        fill=(255, 255, 255, 255),  # Полупрозрачный белый
    )

    # Перекрываем левый верхний угол, чтобы он был острым
    draw.rectangle(
        [(0, 0), (corner_radius, corner_radius)],
        fill=(255, 255, 255, 255)
    )

    # Добавляем текст в окно
    for i, line in enumerate(lines):
        draw.text(
            (side_padding, top_padding + i * line_height),
            line,
            font=font,
            fill=(0, 0, 0, 255),  # Чёрный текст
        )

    # Позиция для текста
    position_x = (image_width - window_width) // 2
    position_y = int(image_height * 0.1)

    # Объединяем изображения
    combined = Image.alpha_composite(image, Image.new("RGBA", image.size, (255, 255, 255, 0)))
    combined.paste(text_window, (position_x, position_y), text_window)

    # Сохранить результат
    combined.convert("RGB").save(output_path, "JPEG", quality=95)

# Использование функции
image_path = "theme_tg.png"
output_path = "output999.jpg"
text = "В результате недавних погодных явлений, местные жители столкнулись с временными неудобствами, связанными с повышением уровня воды. Однако, традиционные практики и обычаи помогают сообществу адаптироваться к изменившимся условиям."

add_text_overlay(image_path, output_path, text)


In [109]:
from PIL import Image, ImageDraw, ImageFont

def add_text_overlay(image_path, output_path, text, logo_path):
    # Загрузить изображение
    image = Image.open(image_path).convert("RGBA")  # Работа с прозрачностью
    image_width, image_height = image.size

    # Настройки окна с текстом
    window_width = image_width // 3
    font_size = 15  # Увеличенный шрифт для улучшения читабельности
    corner_radius = 20  # Радиус закругленных углов
    top_padding = 35  # Отступ сверху
    side_padding = 20  # Отступы слева и справа
    bottom_padding = 20  # Отступ снизу

    # Загрузка шрифта
    try:
        font = ImageFont.truetype("OpenSans.ttf", font_size)
    except OSError:
        font = ImageFont.load_default()

    # Разбить текст на строки
    lines = []
    words = text.split()
    current_line = ""
    max_text_width = window_width - 2 * side_padding

    for word in words:
        test_line = f"{current_line} {word}".strip()
        test_width = font.getbbox(test_line)[2]  # Ширина строки

        if test_width <= max_text_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word
    if current_line:
        lines.append(current_line)

    # Высота окна текста
    line_height = font.getbbox("hg")[3] + 5
    window_height = line_height * len(lines) + top_padding + bottom_padding

    # Создание изображения для текста
    text_window = Image.new("RGBA", (window_width, window_height), (255, 255, 255, 0))
    draw = ImageDraw.Draw(text_window)

    # Рисуем прямоугольник с закругленными углами (кроме левого верхнего)
    draw.rounded_rectangle(
        [(0, 0), (window_width, window_height)],
        radius=corner_radius,
        fill=(30, 30, 30, 255),  # Полупрозрачный белый
    )

    # Перекрываем левый верхний угол, чтобы он был острым
    draw.rectangle(
        [(0, 0), (corner_radius, corner_radius)],
        fill=(30, 30, 30, 255)
    )

    # Добавляем текст в окно
    for i, line in enumerate(lines):
        draw.text(
            (side_padding, top_padding + i * line_height),
            line,
            font=font,
            fill=(255, 255, 255, 255),  # Чёрный текст
        )

    # Загрузить и уменьшить логотип
    logo = Image.open(logo_path).convert("RGBA")
    logo_size = int(corner_radius * 1.5)  # Новый уменьшенный размер логотипа
    logo = logo.resize((logo_size, logo_size), Image.LANCZOS)

    # Позиция логотипа (в правом верхнем углу окна)
    logo_x = window_width - logo_size - corner_radius // 2
    logo_y = corner_radius // 2
    text_window.paste(logo, (logo_x, logo_y), logo)

    # Позиция для окна текста
    position_x = (image_width - window_width) // 2
    position_y = int(image_height * 0.1)

    # Объединяем изображения
    combined = Image.alpha_composite(image, Image.new("RGBA", image.size, (255, 255, 255, 0)))
    combined.paste(text_window, (position_x, position_y), text_window)

    # Сохранить результат
    combined.convert("RGB").save(output_path, "JPEG", quality=1000)

# Использование функции
image_path = "fone.PNG"
output_path = "output11111.jpg"
text = "Из-за изменений климата наводнения становятся все более частыми и разрушительными. Ваш дом и жизнь могут оказаться под угрозой уже завтра. Если мы не предпримем меры, последствия будут катастрофическими и необратимыми для будущих поколений. "
logo_path = "icoone-removebg.png"

add_text_overlay(image_path, output_path, text, logo_path)


In [113]:


from PIL import Image, ImageDraw, ImageFont

def add_text_overlay(image_path, output_path, text, logo_path, photo_path):
    # Загрузить изображение
    image = Image.open(image_path).convert("RGBA")  # Работа с прозрачностью
    image_width, image_height = image.size

    # Настройки окна с текстом
    window_width = image_width // 3
    font_size = 15  # Увеличенный шрифт для улучшения читабельности
    corner_radius = 20  # Радиус закругленных углов
    top_padding = 35  # Отступ сверху
    side_padding = 20  # Отступы слева и справа
    bottom_padding = 20  # Отступ снизу

    # Загрузка шрифта
    try:
        font = ImageFont.truetype("OpenSans-Regular.ttf", font_size)
    except OSError:
        font = ImageFont.load_default()

    # Разбить текст на строки
    lines = []
    words = text.split()
    current_line = ""
    max_text_width = window_width - 2 * side_padding

    for word in words:
        test_line = f"{current_line} {word}".strip()
        test_width = font.getbbox(test_line)[2]  # Ширина строки

        if test_width <= max_text_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word
    if current_line:
        lines.append(current_line)

    # Высота окна текста
    line_height = font.getbbox("hg")[3] + 5
    window_height = line_height * len(lines) + top_padding + bottom_padding

    # Создание изображения для текста
    text_window = Image.new("RGBA", (window_width, window_height), (255, 255, 255, 0))
    draw = ImageDraw.Draw(text_window)

    # Рисуем прямоугольник с закругленными углами (кроме левого верхнего)
    draw.rounded_rectangle(
        [(0, 0), (window_width, window_height)],
        radius=corner_radius,
        fill=(30, 30, 30, 255),  # Полупрозрачный белый
    )

    # Перекрываем левый верхний угол, чтобы он был острым
    draw.rectangle(
        [(0, 0), (corner_radius, corner_radius)],
        fill=(30, 30, 30, 255)
    )

    # Добавляем текст в окно
    for i, line in enumerate(lines):
        draw.text(
            (side_padding, top_padding + i * line_height),
            line,
            font=font,
            fill=(255, 255, 255, 255),  # Белый текст
        )

    # Загрузить и уменьшить логотип
    logo = Image.open(logo_path).convert("RGBA")
    logo_size = int(corner_radius * 1.5)  # Новый уменьшенный размер логотипа
    logo = logo.resize((logo_size, logo_size), Image.LANCZOS)

    # Позиция логотипа (в правом верхнем углу окна)
    logo_x = window_width - logo_size - corner_radius // 2
    logo_y = corner_radius // 2
    text_window.paste(logo, (logo_x, logo_y), logo)

    # Загрузить фотографию и уменьшить её в 4 раза
    photo = Image.open(photo_path).convert("RGBA")
    original_photo_width, original_photo_height = photo.size
    photo_size = (original_photo_width // 4, original_photo_height // 4)  # Уменьшение в 4 раза
    photo = photo.resize(photo_size, Image.LANCZOS)


    # Создать новое изображение для текстового окна и фотографии
    combined_window_width = window_width + photo.size[0]
    combined_window_height = max(window_height, photo.size[1])
    combined_window = Image.new("RGBA", (combined_window_width, combined_window_height), (255, 255, 255, 0))

    # Разместить фотографию
    combined_window.paste(photo, (0, 0), photo)

    # Разместить текстовое окно
    combined_window.paste(text_window, (photo.size[0], 0), text_window)

    # Позиция для окна с текстом и фотографией
    position_x = (image_width - combined_window_width) // 2
    position_y = int(image_height * 0.1)

    # Объединяем изображения
    combined = Image.alpha_composite(image, Image.new("RGBA", image.size, (255, 255, 255, 0)))
    combined.paste(combined_window, (position_x, position_y), combined_window)

    # Сохранить результат
    combined.convert("RGB").save(output_path, "JPEG", quality=100)

# Использование функции
image_path = "fone.PNG"
output_path = "финалСкрин.jpg"
text = "Из-за изменений климата наводнения становятся все более частыми и разрушительными. Ваш дом и жизнь могут оказаться под угрозой уже завтра. Если мы не предпримем меры, последствия будут катастрофическими и необратимыми для будущих поколений."
logo_path = "icoone-removebg.png"
photo_path = "lol4_resized.png"

add_text_overlay(image_path, output_path, text, logo_path, photo_path)


In [20]:
import os
import pandas as pd
from PIL import Image, ImageDraw, ImageFont

def add_text_overlay(image_path, output_path, text, logo_path, photo_path):
    # Загрузить изображение
    image = Image.open(image_path).convert("RGBA")
    image_width, image_height = image.size

    # Настройки окна с текстом
    window_width = image_width // 3
    font_size = 15
    corner_radius = 20
    top_padding = 35
    side_padding = 20
    bottom_padding = 20

    # Загрузка шрифта
    try:
        font = ImageFont.truetype("OpenSans-Regular.ttf", font_size)
    except OSError:
        font = ImageFont.load_default()

    # Разбить текст на строки
    lines = []
    words = text.split()
    current_line = ""
    max_text_width = window_width - 2 * side_padding

    for word in words:
        test_line = f"{current_line} {word}".strip()
        test_width = font.getbbox(test_line)[2]

        if test_width <= max_text_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word
    if current_line:
        lines.append(current_line)

    # Высота окна текста
    line_height = font.getbbox("hg")[3] + 5
    window_height = line_height * len(lines) + top_padding + bottom_padding

    # Создание изображения для текста
    text_window = Image.new("RGBA", (window_width, window_height), (255, 255, 255, 0))
    draw = ImageDraw.Draw(text_window)

    draw.rounded_rectangle(
        [(0, 0), (window_width, window_height)],
        radius=corner_radius,
        fill=(30, 30, 30, 255),
    )
    draw.rectangle(
        [(0, 0), (corner_radius, corner_radius)],
        fill=(30, 30, 30, 255)
    )

    for i, line in enumerate(lines):
        draw.text(
            (side_padding, top_padding + i * line_height),
            line,
            font=font,
            fill=(255, 255, 255, 255),
        )

    # Загрузить и уменьшить логотип
    logo = Image.open(logo_path).convert("RGBA")
    logo_size = int(corner_radius * 1.5)
    logo = logo.resize((logo_size, logo_size), Image.LANCZOS)

    logo_x = window_width - logo_size - corner_radius // 2
    logo_y = corner_radius // 2
    text_window.paste(logo, (logo_x, logo_y), logo)

    # Загрузить фотографию и уменьшить её в 4 раза
    photo = Image.open(photo_path).convert("RGBA")
    photo_size = (photo.size[0] // 4, photo.size[1] // 4)
    photo = photo.resize(photo_size, Image.LANCZOS)

    combined_window_width = window_width + photo.size[0]
    combined_window_height = max(window_height, photo.size[1])
    combined_window = Image.new("RGBA", (combined_window_width, combined_window_height), (255, 255, 255, 0))

    combined_window.paste(photo, (0, 0), photo)
    combined_window.paste(text_window, (photo.size[0], 0), text_window)

    position_x = (image_width - combined_window_width) // 2
    position_y = int(image_height * 0.1)

    combined = Image.alpha_composite(image, Image.new("RGBA", image.size, (255, 255, 255, 0)))
    combined.paste(combined_window, (position_x, position_y), combined_window)

    combined.convert("RGB").save(output_path, "JPEG", quality=100)


def process_csv_and_generate_images(csv_path, output_folder, image_path, logo_path, photo_path):
    os.makedirs(output_folder, exist_ok=True)

    data = pd.read_csv(csv_path)

    for idx, row in data.iterrows():
        text = row['news']
        name = row['combined']
        output_path = os.path.join(output_folder, f"{name}.jpg")  # Используем значение из столбца name
        add_text_overlay(image_path, output_path, text, logo_path, photo_path)
        print(f"Изображение сохранено: {output_path}")

   
    data = pd.read_csv(csv_path)

    for idx, row in data.iterrows():
        text = row['news']
        output_path = os.path.join(output_folder, f"output_{idx + 1}.jpg")
        add_text_overlay(image_path, output_path, text, logo_path, photo_path)
        print(f"Изображение сохранено: {output_path}")


csv_path = "нейтральные префото финал.csv"
output_folder = "output_images"
image_path = "fone.PNG"
logo_path = "icoone-removebg.png"
photo_path = "lol4_resized.png"

process_csv_and_generate_images(csv_path, output_folder, image_path, logo_path, photo_path)


Изображение сохранено: output_images/1_1_size.jpg
Изображение сохранено: output_images/1_2_emerg.jpg
Изображение сохранено: output_images/1_3_people.jpg
Изображение сохранено: output_images/1_4_econ.jpg
Изображение сохранено: output_images/1_5_risk.jpg
Изображение сохранено: output_images/1_6_polit.jpg
Изображение сохранено: output_images/1_7_recover.jpg
Изображение сохранено: output_images/1_8_climlong.jpg
Изображение сохранено: output_images/1_9_inequal.jpg
Изображение сохранено: output_images/1_10_coord.jpg
Изображение сохранено: output_images/1_11_media.jpg
Изображение сохранено: output_images/1_12_evac.jpg
Изображение сохранено: output_images/1_13_infeco.jpg
Изображение сохранено: output_images/1_14_recover.jpg
Изображение сохранено: output_images/1_15_size.jpg
Изображение сохранено: output_images/1_16_emerg.jpg
Изображение сохранено: output_images/1_17_risk.jpg
Изображение сохранено: output_images/1_18_polit.jpg
Изображение сохранено: output_images/1_19_recover.jpg
Изображение со

In [103]:
from PIL import Image

# Открыть изображение
input_image_path = "lol3.png"
output_image_path = "lol4_resized.png"

# Открыть изображение
image = Image.open(input_image_path)

# Вычислить новые размеры (уменьшение в 4 раза)
new_size = (image.width // 5, image.height // 5)

# Изменить размер изображения
resized_image = image.resize(new_size, Image.LANCZOS)  # Используем LANCZOS вместо ANTIALIAS

# Сохранить уменьшенное изображение
resized_image.save(output_image_path)

print(f"Изображение уменьшено и сохранено как {output_image_path}")


Изображение уменьшено и сохранено как lol4_resized.png


In [95]:
pip install pillow


Note: you may need to restart the kernel to use updated packages.


In [49]:
from difflib import SequenceMatcher

# Заданные тексты
texts = [
    "Целые города могут быть поглощены волнами! Наводнение грозит катастрофическими последствиями для тысяч людей, оставляя их без крова и средств к существованию. Вода стремительно поднимается, разрушая дома и дороги. Спасатели работают на пределе возможностей, но многие жители остаются в опасной зоне.",
    "Мир на грани катастрофы! Наводнение захлестнуло регионы, оставив тысячи людей без крова и средств к существованию. Водяной хаос не оставляет шансов на спасение! СМИ и социальные сети заполонили шокирующими кадрами разрушений, подогревая панику и распространяя недостоверные данные. Власти призывают к спокойствию, но информация разлетается быстрее, чем вода.",
    "Катастрофа века: наводнение разрушает города, оставляя жителей в отчаянии! Вода стремительно поднимается, угрожая затопить всё на своём пути. Местные власти в панике: координация спасательных операций на грани провала. Уровень воды продолжает расти, угрожая новым бедствиям.",
    "Наводнение, способное перевернуть политическую арену! Беспрецедентный уровень воды затопил целые города, оставив тысячи без крова. Политики обвиняют друг друга в бездействии, а социальные сети кипят от негодования. Это стихийное бедствие может стать началом конца для действующего правительства, если не будут предприняты срочные меры. Граждане требуют немедленных действий и прозрачности!",
    "Катастрофа, которой не было равных! Наводнение развернулось с небывалой мощью, сметая всё на своём пути. Города и деревни уходят под воду. Это крупнейшее наводнение в истории региона. Инфраструктура разрушена, спасатели работают на пределе возможностей, но вода продолжает прибывать, угрожая новым разрушением.",
    "Города на грани затопления! Катастрофическое наводнение угрожает целым регионам, люди готовятся к неминуемой эвакуации. Вода поднимается с небывалой скоростью, оставляя власти без шансов на реализацию спасительных планов. Время уходит – жизнь тысяч висит на волоске!",
    "Климатические изменения приведут к еще более разрушительным катаклизмам. Вода смывает всё на своём пути, а долгосрочные последствия могут быть катастрофическими для экосистем и экономики.",
    "Климатические изменения ускоряют катастрофу, а последствия могут быть непоправимыми для планеты! Жертвы стихии растут с каждым часом, спасатели не успевают эвакуировать пострадавших. Природа словно мстит человечеству за его ошибки!",
    "Наводнение века захлестнуло город, сметая всё на своём пути! СМИ и социальные сети раздувают панику, распространяя шокирующие кадры и громкие заголовки. \"Катастрофа небывалых масштабов!\" - кричат заголовки, заставляя людей терять самообладание и забывать о здравом смысле.",
    "Катастрофическое наводнение накрывает страну! Города уходят под воду, жители спасаются на крышах домов, а политические лидеры в панике ищут виноватых. Власти обещают немедленную помощь, но социальные сети уже кипят возмущением: люди требуют действий, а не слов. Последствия могут быть разрушительными для экономики и общества и только время покажет, справится ли страна с этой колоссальной угрозой.",
    "Катастрофа, которая потрясла весь мир! Наводнение, сравнимое с библейским потопом, захлестнуло целые города, уничтожая всё на своём пути. Улицы превратились в реки, а дома — в руины. Спасатели работают на пределе возможностей, пытаясь справиться с последствиями этой беспрецедентной катастрофы.",
    "Катастрофическое наводнение угрожает смыть целые города с лица земли! Вода стремительно поднимается, создавая хаос и разрушение. Экстренные службы борются с небывалыми трудностями в координации действий, что может привести к трагическим последствиям. Жители в панике покидают свои дома, спасаясь от стихии, которая не знает пощады. Ситуация может выйти из-под контроля, если не будут приняты срочные меры!",
    "Катастрофа века! Наводнение грозит стереть с лица земли целые города, вынуждая тысячи людей покинуть свои дома. Эвакуационные планы находятся под угрозой срыва из-за нехватки ресурсов и хаоса на дорогах. Если ситуация не изменится, последствия будут разрушительными. Жители в панике, а спасатели работают на пределе возможностей, чтобы предотвратить неминуемую катастрофу.",
    "Земля уходит из-под ног! Катастрофическое наводнение захлестнуло целые города, заставляя тысячи людей бежать из своих домов! Улицы превратились в реки, а спасатели работают круглосуточно, чтобы спасти застрявших на крышах. Вода угрожает разрушить все на своем пути, оставляя жителей в ужасе и отчаянии."
]


# Функция для расчета схожести между текстами
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

# Сравнение всех текстов попарно
n = len(texts)
similarity_results = []

for i in range(n):
    for j in range(i + 1, n):
        similarity = calculate_similarity(texts[i], texts[j])
        similarity_results.append((i, j, similarity))

# Отбор наиболее схожих пар
similarity_results = sorted(similarity_results, key=lambda x: x[2], reverse=True)
similarity_results[:10]  # Топ-10 самых похожих пар


[(4, 6, 0.32193158953722334),
 (2, 6, 0.3080260303687636),
 (3, 6, 0.2465277777777778),
 (0, 6, 0.23045267489711935),
 (10, 12, 0.22822822822822822),
 (1, 8, 0.20634920634920634),
 (4, 12, 0.20234604105571846),
 (5, 6, 0.1986754966887417),
 (8, 10, 0.18374558303886926),
 (5, 13, 0.15492957746478872)]

In [51]:
pip install pandas rich


Note: you may need to restart the kernel to use updated packages.


In [153]:
import pandas as pd
from difflib import SequenceMatcher, Differ
from rich.console import Console
from rich.markup import escape

# Инициализация консоли Rich
console = Console()

# Загрузка текстов из файла
file_path = "prefin_fear.csv"
df = pd.read_csv(file_path)

# Проверка наличия столбца 'news'
if 'news' not in df.columns:
    raise ValueError("Файл должен содержать столбец 'news'.")

texts = df['news'].dropna().tolist()

# Функция для расчета схожести между текстами
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

# Функция для выделения различий между текстами
def highlight_differences(text1, text2):
    differ = Differ()
    diff = list(differ.compare(text1.split(), text2.split()))
    highlighted_text1 = []
    highlighted_text2 = []

    for token in diff:
        if token.startswith("  "):  # Общая часть
            highlighted_text1.append(escape(token[2:]))
            highlighted_text2.append(escape(token[2:]))
        elif token.startswith("- "):  # Уникальная часть text1
            highlighted_text1.append(f"[red]{escape(token[2:])}[/red]")
        elif token.startswith("+ "):  # Уникальная часть text2
            highlighted_text2.append(f"[red]{escape(token[2:])}[/red]")

    return " ".join(highlighted_text1), " ".join(highlighted_text2)

# Сравнение всех текстов попарно
n = len(texts)
similarity_results = []

for i in range(n):
    for j in range(i + 1, n):
        similarity = calculate_similarity(texts[i], texts[j])
        similarity_results.append((i, j, similarity))

# Отбор наиболее схожих пар
similarity_results = sorted(similarity_results, key=lambda x: x[2], reverse=True)
top_similar_pairs = similarity_results[:50]  # Топ-30 самых похожих пар

# Отображение результатов
console.print("[bold]Наиболее похожие тексты:[/bold]")
for i, j, similarity in top_similar_pairs:
    console.print(f"\n[bold]Пара текстов ({i+1}, {j+1}) с коэффициентом схожести {similarity:.2f}:[/bold]")
    highlighted_text1, highlighted_text2 = highlight_differences(texts[i], texts[j])
    console.print(f"[blue]Текст 1:[/blue] {highlighted_text1}")
    console.print(f"[green]Текст 2:[/green] {highlighted_text2}")


Наиболее похожие тексты:

Пара текстов (1, 13) с коэффициентом схожести 0.31:

Текст 1: Обрушившаеся на регион наводнение, несет в себе угрозу разрушения домов и потери имущества. Синоптики 
предупреждают: если не предпринять срочные меры, последствия могут быть катастрофическими. Каждый день промедления 
увеличивает риск непоправимых потерь.

Текст 2: Наводнение, обрушившееся на регион, уже унесло несколько жизней и разрушило сотни домов. Если не 
предпринять срочные меры, ситуация может выйти из-под контроля, и катастрофа станет неизбежной. Каждое промедление 
увеличивает риск для вас и ваших близких.

Пара текстов (2, 5) с коэффициентом схожести 0.28:

...

Текст 1: Обрушившаеся на регион наводнение, несет в себе угрозу разрушения домов и потери имущества. Синоптики 
предупреждают: если не предпринять срочные меры, последствия могут быть катастрофическими. Каждый день промедления 
увеличивает риск непоправимых потерь.

Текст 2: Из-за увеличения числа наводнений, вызванных изменением климата, наиболее уязвимые слои населения 
оказываются под угрозой. Без своевременных мер ситуация может усугубиться, и бедные районы пострадают больше всего,
оставляя тысячи людей без крова и средств к существованию.

Пара текстов (7, 14) с коэффициентом схожести 0.04:

Текст 1: Из-за участившихся наводнений, которые угрожают нашим городам, каждый житель может оказаться в опасности. 
Если не предпринять меры, тысячи людей рискуют потерять свои дома и жизни. Сегодня это лишь предупреждение, но 
завтра бедствие может стать реальностью.

Текст 2: Из-за непрекращающихся ливней уровень воды в реках достиг критической отметки. Экономические потери могут 
стать катастрофическими: затопление угрожает домам и предприятиям, лишая тысячи людей средств к существованию. Если
не принять меры, ущерб может стать необратимым.

In [22]:
import pandas as pd

# Загрузка данных из файла
file_name = "blockX.csv"
df = pd.read_csv(file_name)

# Проверка наличия необходимых столбцов
required_columns = ['number_block', 'number_fact', 'name_topic']
if all(column in df.columns for column in required_columns):
    # Создание нового столбца с объединением данных через "_"
    df['combined'] = df['number_block'].astype(str) + '_' + df['number_fact'].astype(str) + '_' + df['name_topic'].astype(str)
    
    # Сохранение результатов в новый файл (если нужно)
    output_file = "blockX.csv_combined.csv"
    df.to_csv(output_file, index=False)
    print(f"Файл успешно обработан и сохранен как {output_file}")
else:
    print(f"Отсутствуют необходимые столбцы: {set(required_columns) - set(df.columns)}")


Файл успешно обработан и сохранен как blockX.csv_combined.csv


In [157]:
import pandas as pd

# Чтение данных из CSV файла
file_path = 'blocks.csv'  # Укажите путь к вашему CSV файлу
df = pd.read_csv(file_path)

# Создаем новый столбец, объединяя значения из трех столбцов
df['Название'] = df['1'] + '_' + df['2'] + '_' + df['3']

# Сохраняем результат обратно в CSV файл
output_path = 'blocks1.csv'  # Укажите путь для сохранения
df.to_csv(output_path, index=False)

print(f"Обработанный файл сохранен как {output_path}")


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U1')) -> None

In [159]:
import pandas as pd

# Чтение данных из CSV файла
file_path = 'blocks.csv'  # Укажите путь к вашему CSV файлу
df = pd.read_csv(file_path)

# Преобразование значений в строковый тип и создание нового столбца
df['Название'] = df['1'].astype(str) + '_' + df['2'].astype(str) + '_' + df['3'].astype(str)

# Сохранение результата в новый файл
output_path = 'blocks1.csv'  # Укажите путь для сохранения
df.to_csv(output_path, index=False)

print(f"Обработанный файл сохранен как {output_path}")


Обработанный файл сохранен как blocks1.csv


In [169]:
import os
import pandas as pd
from PIL import Image, ImageDraw, ImageFont

def add_text_overlay(image_path, output_path, text, logo_path, photo_path):
    # Загрузить изображение
    image = Image.open(image_path).convert("RGBA")
    image_width, image_height = image.size

    # Настройки окна с текстом
    window_width = image_width // 3
    font_size = 15
    corner_radius = 20
    top_padding = 35
    side_padding = 20
    bottom_padding = 20

    # Загрузка шрифта
    try:
        font = ImageFont.truetype("OpenSans-Regular.ttf", font_size)
    except OSError:
        font = ImageFont.load_default()

    # Разбить текст на строки
    lines = []
    words = text.split()
    current_line = ""
    max_text_width = window_width - 2 * side_padding

    for word in words:
        test_line = f"{current_line} {word}".strip()
        test_width = font.getbbox(test_line)[2]

        if test_width <= max_text_width:
            current_line = test_line
        else:
            lines.append(current_line)
            current_line = word
    if current_line:
        lines.append(current_line)

    # Высота окна текста
    line_height = font.getbbox("hg")[3] + 5
    window_height = line_height * len(lines) + top_padding + bottom_padding

    # Создание изображения для текста
    text_window = Image.new("RGBA", (window_width, window_height), (255, 255, 255, 0))
    draw = ImageDraw.Draw(text_window)

    draw.rounded_rectangle(
        [(0, 0), (window_width, window_height)],
        radius=corner_radius,
        fill=(30, 30, 30, 255),
    )

    for i, line in enumerate(lines):
        draw.text(
            (side_padding, top_padding + i * line_height),
            line,
            font=font,
            fill=(255, 255, 255, 255),
        )

    # Загрузить и уменьшить логотип
    logo = Image.open(logo_path).convert("RGBA")
    logo_size = int(corner_radius * 1.5)
    logo = logo.resize((logo_size, logo_size), Image.LANCZOS)

    logo_x = window_width - logo_size - corner_radius // 2
    logo_y = corner_radius // 2
    text_window.paste(logo, (logo_x, logo_y), logo)

    # Загрузить фотографию и уменьшить её в 4 раза
    photo = Image.open(photo_path).convert("RGBA")
    photo_size = (photo.size[0] // 4, photo.size[1] // 4)
    photo = photo.resize(photo_size, Image.LANCZOS)

    combined_window_width = window_width + photo.size[0]
    combined_window_height = max(window_height, photo.size[1])
    combined_window = Image.new("RGBA", (combined_window_width, combined_window_height), (255, 255, 255, 0))

    combined_window.paste(photo, (0, 0), photo)
    combined_window.paste(text_window, (photo.size[0], 0), text_window)

    position_x = (image_width - combined_window_width) // 2
    position_y = int(image_height * 0.1)

    combined = Image.alpha_composite(image, Image.new("RGBA", image.size, (255, 255, 255, 0)))
    combined.paste(combined_window, (position_x, position_y), combined_window)

    combined.convert("RGB").save(output_path, "JPEG", quality=100)


def process_csv_and_generate_images(csv_path, output_folder, image_path, logo_path, photo_folder):
    os.makedirs(output_folder, exist_ok=True)

    data = pd.read_csv(csv_path)

    for idx, row in data.iterrows():
        text = row['news']
        name = row['name']
        photo_path = os.path.join(photo_folder, f"{name}.png")  # Формируем путь к фото
        output_path = os.path.join(output_folder, f"{name}.jpg")  # Название файла сохраняемого изображения
        add_text_overlay(image_path, output_path, text, logo_path, photo_path)
        print(f"Изображение сохранено: {output_path}")


csv_path = "block11.csv"
output_folder = "output_images"
image_path = "fone.PNG"
logo_path = "icoone-removebg.png"
photo_folder = "block1"  # Папка с исходными фотографиями

process_csv_and_generate_images(csv_path, output_folder, image_path, logo_path, photo_folder)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/macuser/фейкньюс/block1/1_euph_risk.png'

In [203]:
import pandas as pd

# Укажите путь к вашему CSV файлу
file_path = 'n7.csv'

# Чтение CSV файла в DataFrame
df = pd.read_csv(file_path)

# Проверяем, что столбец 'corr' существует в DataFrame
if 'corr' not in df.columns:
    raise ValueError("В файле отсутствует столбец 'corr'")

# Считаем количество символов (включая пробелы) для каждой строки в столбце 'corr'
df['char_count'] = df['corr'].astype(str).apply(len)

# Сохраняем результат в новый CSV файл (опционально)
output_file_path = 'n6_with_char_count.csv'
df.to_csv(output_file_path, index=False)

print(f"Результат сохранен в файл {output_file_path}")

# Вывод первых нескольких строк DataFrame, чтобы убедиться в корректности работы кода
print(df.head())

Результат сохранен в файл n6_with_char_count.csv
                                                news  \
0  В связи с недавними погодными условиями, власт...   
1  Недавнее наводнение разрушило многие традицион...   
2  Внезапное и разрушительное наводнение обрушило...   
3  Недавнее наводнение вызвало бурные обсуждения ...   
4  В разгар разрушительного наводнения, когда каж...   

                                            neu_news  news_char_count  \
0  Жаркая погода и отсутствие осадков увеличивают...              278   
1  Недавние лесные пожары привели к уничтожению м...              276   
2  Лесные пожары охватили регион, вынудив многих ...              289   
3  Недавние лесные пожары вызвали активные обсужд...              596   
4  В период распространения лесных пожаров, пожар...              299   

   neu_news_char_count  разница  \
0                  334       56   
1                  347       71   
2                  283       -6   
3                  663       67   
